# Objectives
1. ETA Prediction = customer satisfaction
2. Productivity = reducing operating costs
3. Anomalies = preventing hidden losses
4. Problematic areas = strategic planning of the logistics network

# Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

# EDA Part 1 - Understanding the Dataset

In [5]:
df = pd.read_csv(r'../data/raw/amazon_delivery.csv')
df.head()

,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,motorcycle,Urban,120,Clothing
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,scooter,Metropolitian,165,Electronics
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,motorcycle,Urban,130,Sports
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,motorcycle,Metropolitian,105,Cosmetics
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,scooter,Metropolitian,150,Toys


In [6]:
# order_date, order_time and pickpu_time are string
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43739 entries, 0 to 43738
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Order_ID         43739 non-null  object 
 1   Agent_Age        43739 non-null  int64  
 2   Agent_Rating     43685 non-null  float64
 3   Store_Latitude   43739 non-null  float64
 4   Store_Longitude  43739 non-null  float64
 5   Drop_Latitude    43739 non-null  float64
 6   Drop_Longitude   43739 non-null  float64
 7   Order_Date       43739 non-null  object 
 8   Order_Time       43739 non-null  object 
 9   Pickup_Time      43739 non-null  object 
 10  Weather          43648 non-null  object 
 11  Traffic          43739 non-null  object 
 12  Vehicle          43739 non-null  object 
 13  Area             43739 non-null  object 
 14  Delivery_Time    43739 non-null  int64  
 15  Category         43739 non-null  object 
dtypes: float64(5), int64(2), object(9)
memory usage: 5.3+ MB


In [7]:
# missing values already spotted
df.describe()

,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Delivery_Time
count,43739.000000,43685.000000,43739.000000,43739.000000,43739.000000,43739.000000,43739.000000
mean,29.567137,4.633780,17.210960,70.661177,17.459031,70.821842,124.905645
std,5.815155,0.334716,7.764225,21.475005,7.342950,21.153148,51.915451
min,15.000000,1.000000,-30.902872,-88.366217,0.010000,0.010000,10.000000
25%,25.000000,4.500000,12.933298,73.170283,12.985996,73.280000,90.000000
50%,30.000000,4.700000,18.551440,75.898497,18.633626,76.002574,125.000000
75%,35.000000,4.900000,22.732225,78.045359,22.785049,78.104095,160.000000
max,50.000000,6.000000,30.914057,88.433452,31.054057,88.563452,270.000000


In [8]:
df.isnull().sum()

Order_ID            0
Agent_Age           0
Agent_Rating       54
Store_Latitude      0
Store_Longitude     0
Drop_Latitude       0
Drop_Longitude      0
Order_Date          0
Order_Time          0
Pickup_Time         0
Weather            91
Traffic             0
Vehicle             0
Area                0
Delivery_Time       0
Category            0
dtype: int64

In [9]:
# considering that there are NaN values and considering that they represent less than 1% of the table, the lines are going to be dropped
df = df.dropna()

In [10]:
# confirming that there are no NaN values
df.isnull().sum().sum()

np.int64(0)

In [11]:
# order_date, order_time and pickup_time datatypes are wrong
# in order to solve this problem I wanna convert the 2 time columns to decimal and order date from string to datetime type
df['Order_Date'] = pd.to_datetime(df['Order_Date'])

for col in ["Order_Time", "Pickup_Time"]:
    df[col] = pd.to_timedelta(df[col]).dt.total_seconds() / 3600

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43594 entries, 0 to 43738
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Order_ID         43594 non-null  object        
 1   Agent_Age        43594 non-null  int64         
 2   Agent_Rating     43594 non-null  float64       
 3   Store_Latitude   43594 non-null  float64       
 4   Store_Longitude  43594 non-null  float64       
 5   Drop_Latitude    43594 non-null  float64       
 6   Drop_Longitude   43594 non-null  float64       
 7   Order_Date       43594 non-null  datetime64[ns]
 8   Order_Time       43594 non-null  float64       
 9   Pickup_Time      43594 non-null  float64       
 10  Weather          43594 non-null  object        
 11  Traffic          43594 non-null  object        
 12  Vehicle          43594 non-null  object        
 13  Area             43594 non-null  object        
 14  Delivery_Time    43594 non-null  int64     

In [12]:
# now we have 6 objects, 9 numeric e 1 datetime columns
df.head()

,Order_ID,Agent_Age,Agent_Rating,Store_Latitude,Store_Longitude,Drop_Latitude,Drop_Longitude,Order_Date,Order_Time,Pickup_Time,Weather,Traffic,Vehicle,Area,Delivery_Time,Category
0,ialx566343618,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11.50,11.750000,Sunny,High,motorcycle,Urban,120,Clothing
1,akqg208421122,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19.75,19.833333,Stormy,Jam,scooter,Metropolitian,165,Electronics
2,njpu434582536,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,8.50,8.750000,Sandstorms,Low,motorcycle,Urban,130,Sports
3,rjto796129700,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18.00,18.166667,Sunny,Medium,motorcycle,Metropolitian,105,Cosmetics
4,zguw716275638,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13.50,13.750000,Cloudy,High,scooter,Metropolitian,150,Toys


# EDA Part 2 - Understanding the Dataset

## Analysing object type

In [24]:
def profile_object_columns(df):
    """
    Profile analysis for object (categorical) columns.
    Prints unique values and value counts > 1 with clear separation.
    """
    object_columns = df.select_dtypes(include=['object']).columns
    
    for column in object_columns:
        print(f"\n{'='*60}")
        print(f"Column: {column}")
        print(f"{'='*60}\n")
        
        # Unique values
        uniques = df[column].unique()
        print(f"Unique values ({len(uniques)}):")
        print(uniques)
        print(f"\n{'-'*60}\n")
        
        # Value counts sorted ascending, only values appearing more than once
        counts = df[column].value_counts()
        repeated_counts = counts[counts > 1].sort_values()
        if not repeated_counts.empty:
            print(f"Value counts (appearing more than once, ascending):")
            print(repeated_counts.to_string())
        else:
            print("No repeated values.")
        
        print(f"\n{'='*60}\n")


profile_object_columns(df)


Column: Order_ID

Unique values (43594):
['ialx566343618' 'akqg208421122' 'njpu434582536' ... 'xnek760674819'
 'cynl434665991' 'nsyz997960170']

------------------------------------------------------------

No repeated values.



Column: Weather

Unique values (6):
['Sunny' 'Stormy' 'Sandstorms' 'Cloudy' 'Fog' 'Windy']

------------------------------------------------------------

Value counts (appearing more than once, ascending):
Weather
Sunny         7067
Windy         7211
Sandstorms    7238
Cloudy        7277
Stormy        7366
Fog           7435



Column: Traffic

Unique values (4):
['High ' 'Jam ' 'Low ' 'Medium ']

------------------------------------------------------------

Value counts (appearing more than once, ascending):
Traffic
High        4294
Medium     10614
Jam        13710
Low        14976



Column: Vehicle

Unique values (3):
['motorcycle ' 'scooter ' 'van']

------------------------------------------------------------

Value counts (appearing more than once, as

### Critical Points
* Traffic: “High” = 4294, “Low” = 14976 → significant imbalance (~3.5x).
* Vehicle: “van” = 3518, “motorcycle” = 25486 → notable imbalance (~7x).
* Area: “Semi-Urban” = 152, “Metropolitian” = 32592 → extreme imbalance (~215x).
* Impact: These columns can bias ML models or statistical analyses; consider resampling, grouping categories, or weighting.

## Analysing numeric type

In [18]:
def profile_analyser(df):
    """
    Complete profile analysis of numerical columns
    """
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    
    profile = {}
    
    for column in numeric_columns:
        profile[column] = {
            # Basic statistics
            'count': df[column].count(),
            'missing': df[column].isnull().sum(),
            'missing_pct': (df[column].isnull().sum() / len(df)) * 100,
            
            # Central tendency
            'mean': df[column].mean(),
            'median': df[column].median(),
            'mode': df[column].mode().iloc[0] if not df[column].mode().empty else None,
            
            # Dispersion
            'std': df[column].std(),
            'var': df[column].var(),
            'min': df[column].min(),
            'max': df[column].max(),
            'range': df[column].max() - df[column].min(),
            
            # Percentiles
            'q25': df[column].quantile(0.25),
            'q75': df[column].quantile(0.75),
            'iqr': df[column].quantile(0.75) - df[column].quantile(0.25),
            
            # Skewness and kurtosis
            'skewness': df[column].skew(),
            'kurtosis': df[column].kurtosis(),
            
            # Unique values
            'unique_count': df[column].nunique(),
            'unique_pct': (df[column].nunique() / df[column].count()) * 100,
            
            # Outliers (IQR method)
            'outliers_count': len(detect_outliers_iqr(df[column]))
        }
    
    return pd.DataFrame(profile).T

def detect_outliers_iqr(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return series[(series < lower_bound) | (series > upper_bound)]

In [25]:
numeric_analysis = profile_analyser(df)
numeric_analysis

,count,missing,missing_pct,mean,median,mode,std,var,min,max,range,q25,q75,iqr,skewness,kurtosis,unique_count,unique_pct,outliers_count
Agent_Age,43594.0,0.0,0.0,29.555719,30.000000,35.000000,5.760689,33.185538,20.000000,39.000000,19.000000,25.000000,35.000000,10.000000,-0.014471,-1.208368,20.0,0.045878,0.0
Agent_Rating,43594.0,0.0,0.0,4.635287,4.700000,4.800000,0.313827,0.098487,2.500000,5.000000,2.500000,4.500000,4.900000,0.400000,-1.793118,5.137553,26.0,0.059641,1132.0
Store_Latitude,43594.0,0.0,0.0,17.244769,18.554382,0.000000,7.690005,59.136178,-30.902872,30.914057,61.816929,12.933298,22.732225,9.798927,-1.055690,2.353475,489.0,1.121714,151.0
Store_Longitude,43594.0,0.0,0.0,70.768898,75.898497,0.000000,21.128773,446.425034,0.000000,88.433452,88.433452,73.170283,78.045359,4.875076,-2.954381,7.090905,389.0,0.892325,4301.0
Drop_Latitude,43594.0,0.0,0.0,17.461540,18.633934,0.130000,7.338199,53.849159,0.010000,31.054057,31.044057,12.986054,22.785049,9.798995,-0.700486,0.260856,4365.0,10.012846,0.0
Drop_Longitude,43594.0,0.0,0.0,70.832489,76.002471,0.130000,21.128940,446.432124,0.010000,88.563452,88.553452,73.280000,78.102309,4.822309,-2.954371,7.090861,4365.0,10.012846,6468.0
Order_Time,43594.0,0.0,0.0,17.917196,19.250000,21.916667,4.841461,23.439742,0.000000,23.916667,23.916667,15.416667,21.583333,6.166667,-1.046237,0.599608,176.0,0.403725,429.0
Pickup_Time,43594.0,0.0,0.0,17.629523,19.166667,21.500000,5.358668,28.715328,0.000000,23.916667,23.916667,14.583333,21.583333,7.000000,-1.263466,1.322988,193.0,0.442721,1254.0
Delivery_Time,43594.0,0.0,0.0,124.916433,125.000000,130.000000,51.941975,2697.968815,10.000000,270.000000,260.000000,90.000000,160.000000,70.000000,0.188318,-0.262140,89.0,0.204157,79.0


In [26]:
numeric_analysis.to_csv(r'../data/processed/profile_analyser_numeric_columns.csv')

### Critical Points

* Store_Longitude and Drop_Longitude have 10-15% outliers – this can break your model.
* Very skewed distributions in coordinates – robust normalization is required.
* Agent_Rating is concentrated at high values – consider converting it into categories.

# Correlations

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43594 entries, 0 to 43738
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Order_ID         43594 non-null  object        
 1   Agent_Age        43594 non-null  int64         
 2   Agent_Rating     43594 non-null  float64       
 3   Store_Latitude   43594 non-null  float64       
 4   Store_Longitude  43594 non-null  float64       
 5   Drop_Latitude    43594 non-null  float64       
 6   Drop_Longitude   43594 non-null  float64       
 7   Order_Date       43594 non-null  datetime64[ns]
 8   Order_Time       43594 non-null  float64       
 9   Pickup_Time      43594 non-null  float64       
 10  Weather          43594 non-null  object        
 11  Traffic          43594 non-null  object        
 12  Vehicle          43594 non-null  object        
 13  Area             43594 non-null  object        
 14  Delivery_Time    43594 non-null  int64     

In [31]:
df.columns

Index(['Order_ID', 'Agent_Age', 'Agent_Rating', 'Store_Latitude',
       'Store_Longitude', 'Drop_Latitude', 'Drop_Longitude', 'Order_Date',
       'Order_Time', 'Pickup_Time', 'Weather', 'Traffic', 'Vehicle', 'Area',
       'Delivery_Time', 'Category'],
      dtype='object')

In [34]:
# Heatmap de correlações
df_for_corr = df[['Agent_Age', 'Agent_Rating', 'Store_Latitude',
       'Store_Longitude', 'Drop_Latitude', 'Drop_Longitude', 'Order_Date',
       'Order_Time', 'Pickup_Time']]
correlation_matrix = df_for_corr.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')

NameError: name 'sns' is not defined